### Importing all the necessary libraries of NLP, keras, ML

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import re
import spacy

In [3]:
# importing tf-keras libraries for using word-embeddings
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

In [4]:
nlp = spacy.load("en_core_web_sm")

### Loading the dataset from local machine

In [8]:
train = pd.read_csv(r'E:Kaggle Playground\Disaster prediction using Tweets\train.csv')
test = pd.read_csv(r'E:\Kaggle Playground\Disaster prediction using Tweets\test.csv')

In [9]:
train.shape

(7613, 5)

In [10]:
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [11]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

### Preprocessing the data 
1. Removing some columns (like keyword, location which either contain lot of NaN values or doesn't provide relevant information).
2. Removing all the links, stopwords, punctuations, extra spaces present in the data.
3. Doing .lower() to bring data in same case.
4. Creating word corpus from data, i.e., converting strings into list of words for feeding into Neural Network

In [ ]:
train.drop(columns='location', inplace=True)
test.drop(columns='location', inplace=True)

In [17]:
# ACCEPTS STRING => RETURNS STRING
def link_hashtag_remover(text):
  # reference : https://stackoverflow.com/questions/8376691/how-to-remove-hashtag-user-link-of-a-tweet-using-regular-expression
  # pattern = 'http:[\/]{2}[a-z].[a-z]*[\/][a-zA-Z0-9]*|#[a-zA-Z0-9]*|@[a-zA-Z0-9]*'

  preprocessed = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
  #re.sub() is used to replace a given pattern with a replacement pattern
  return preprocessed


# ACCEPTS STRING => RETURNS LIST
def stopwords_punct_remover(text):
  doc = nlp(text)

  preprocessed = []

  for token in doc:
    if(token.is_stop == False):
      if(token.pos_ not in ["PUNCT", "SPACE"]):
        # print(token.text)
        preprocessed.append(token.text)

  return preprocessed
  #return preprocessed

# ACCEPTS LIST => RETURNS STRING
def lemmatizer(text):
  text = " ".join(text)
  str1 = ""
  doc = nlp(text)
  for token in doc:
    str1 += token.lemma_
    str1 += " "
  return str1

In [18]:
def pipeline(text):
  text1 = link_hashtag_remover(text)
  text2 = stopwords_punct_remover(text1)
  text3 = lemmatizer(text2)

  return text3

In [19]:
def create_corpus(df):
    corpus=[]
    for tweet in df['input']:
        words = [] # list of words
        doc = nlp(tweet)
        for token in doc:
            words.append(token.text.lower())
        corpus.append(words)
    return corpus # list of list of words, i.e., list of sentences

In [20]:
train['input'] = train['text'].map(lambda x:pipeline(x))

In [22]:
train['input']

0                   deed Reason earthquake ALLAH Forgive 
1                  forest fire near La Ronge Sask Canada 
2       resident ask shelter place notify officer evac...
3       13 000 people receive wildfire evacuation orde...
4       got send photo Ruby Alaska smoke wildfire pour...
                              ...                        
7608        giant crane hold bridge collapse nearby home 
7609    ahrary control wild fire California northern s...
7610               M1 94 01 04 utc 5 km S Volcano Hawaii 
7611    Police investigate e bike collide car Little P...
7612    late Homes Razed Northern California Wildfire ...
Name: input, Length: 7613, dtype: object

In [23]:
corpus = create_corpus(train)

In [24]:
len(corpus)

7613

In [25]:
print(corpus[0])

['deed', 'reason', 'earthquake', 'allah', 'forgive']


### Word Embeddings using GloVe
1. Mapping each word from glove with its suitable numerical word vector of dimensions 50 X 1.
2. post padding to make each sentence of equal length by adding 0's at the end.
3. Create embedding and convert the text corpus into numerical data and ignoring the words which are not present in the dictionary.

In [27]:
# using GloVe embeddings to map each word with their vectors
embeddings_dict = {}
with open(r"E:\Glove\glove.6B.50d.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [28]:
MAX_LEN=50
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences = tokenizer_obj.texts_to_sequences(corpus)

tweet_pad = pad_sequences(sequences, maxlen=MAX_LEN, truncating='post', padding='post')
# truncating -> remove values from the back if extra words are present.

In [31]:
tweet_pad[0]

array([3767,  447,  197, 1322, 1796,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [32]:
# maps each unique word to index, not necessarily in alphabetical order
# indexing will start from 1 and not from 0
word_index = tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 13182


In [34]:
tokenizer_obj.word_index

{'s': 1,
 't': 2,
 'like': 3,
 'm': 4,
 'fire': 5,
 'amp': 6,
 'new': 7,
 '2': 8,
 'go': 9,
 'people': 10,
 'news': 11,
 'don': 12,
 'year': 13,
 'video': 14,
 'kill': 15,
 'burn': 16,
 'crash': 17,
 'disaster': 18,
 'emergency': 19,
 'body': 20,
 'come': 21,
 '3': 22,
 'day': 23,
 'bomb': 24,
 'look': 25,
 'police': 26,
 'man': 27,
 'get': 28,
 'attack': 29,
 'know': 30,
 'u': 31,
 'time': 32,
 'flood': 33,
 'storm': 34,
 'family': 35,
 'home': 36,
 'love': 37,
 'california': 38,
 'think': 39,
 '1': 40,
 'good': 41,
 'suicide': 42,
 'watch': 43,
 'building': 44,
 'want': 45,
 'train': 46,
 'say': 47,
 'collapse': 48,
 'car': 49,
 'death': 50,
 'world': 51,
 'rt': 52,
 '5': 53,
 'work': 54,
 'nuclear': 55,
 'cause': 56,
 'life': 57,
 'today': 58,
 '4': 59,
 'let': 60,
 'dead': 61,
 'need': 62,
 'accident': 63,
 'scream': 64,
 'war': 65,
 'old': 66,
 'live': 67,
 'd': 68,
 'gt': 69,
 'hiroshima': 70,
 'wreck': 71,
 'destroy': 72,
 'drown': 73,
 'way': 74,
 'help': 75,
 'fear': 76,
 've'

In [36]:
num_words = len(word_index) + 1
embedding_matrix=np.zeros((num_words,50)) #contains word vectors of size 50 for all words

for word,i in word_index.items():
    if i > num_words:
        continue
    
    emb_vec=embeddings_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

In [37]:
embedding_matrix.shape # since there are 13182 words each of size 50 dimension

(13183, 50)

### Making Neural Network model for Classification of tweet, which contains embedding layer,  dropout layer, LSTM layer

In [38]:
model=Sequential()

embedding = Embedding(num_words, 50, embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN, trainable=True)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy', optimizer=optimzer, metrics=['accuracy'])

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            659150    
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 50)           0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 688,655
Trainable params: 688,655
Non-trainable params: 0
_________________________________________________________________


In [40]:
tweet_pad

array([[3767,  447,  197, ...,    0,    0,    0],
       [ 132,    5,  171, ...,    0,    0,    0],
       [1592,  485, 1593, ...,    0,    0,    0],
       ...,
       [2332, 2006,  544, ...,    0,    0,    0],
       [  26,  783,  425, ...,    0,    0,    0],
       [ 141,  742,  795, ...,    0,    0,    0]])

In [41]:
train['input']

0                   deed Reason earthquake ALLAH Forgive 
1                  forest fire near La Ronge Sask Canada 
2       resident ask shelter place notify officer evac...
3       13 000 people receive wildfire evacuation orde...
4       got send photo Ruby Alaska smoke wildfire pour...
                              ...                        
7608        giant crane hold bridge collapse nearby home 
7609    ahrary control wild fire California northern s...
7610               M1 94 01 04 utc 5 km S Volcano Hawaii 
7611    Police investigate e bike collide car Little P...
7612    late Homes Razed Northern California Wildfire ...
Name: input, Length: 7613, dtype: object

In [42]:
print(word_index['deed'])
print(word_index['reason'])

3767
447


### Splitting the data into test data and train data using train_test_split function from sklearn library

In [43]:
train_data = tweet_pad[:train.shape[0]]

In [44]:
x_train,x_test,y_train,y_test=train_test_split(train_data,train['target'].values,test_size=0.15)
print('Shape of train',x_train.shape)
print("Shape of Validation ",x_test.shape)

Shape of train (6471, 50)
Shape of Validation  (1142, 50)


In [46]:
train_data[0]

array([3767,  447,  197, 1322, 1796,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

### Training the model

In [45]:
history = model.fit(x_train, y_train, batch_size=4, epochs=15, validation_data=(x_test,y_test),verbose=2)

Epoch 1/15
1618/1618 - 60s - loss: 0.6874 - accuracy: 0.5693 - val_loss: 0.6452 - val_accuracy: 0.5963 - 60s/epoch - 37ms/step
Epoch 2/15
1618/1618 - 57s - loss: 0.5791 - accuracy: 0.7166 - val_loss: 0.5443 - val_accuracy: 0.7662 - 57s/epoch - 35ms/step
Epoch 3/15
1618/1618 - 63s - loss: 0.5463 - accuracy: 0.7432 - val_loss: 0.5269 - val_accuracy: 0.7653 - 63s/epoch - 39ms/step
Epoch 4/15
1618/1618 - 61s - loss: 0.5275 - accuracy: 0.7608 - val_loss: 0.5138 - val_accuracy: 0.7785 - 61s/epoch - 37ms/step
Epoch 5/15
1618/1618 - 56s - loss: 0.5191 - accuracy: 0.7578 - val_loss: 0.5069 - val_accuracy: 0.7767 - 56s/epoch - 34ms/step
Epoch 6/15
1618/1618 - 56s - loss: 0.5180 - accuracy: 0.7640 - val_loss: 0.5061 - val_accuracy: 0.7741 - 56s/epoch - 34ms/step
Epoch 7/15
1618/1618 - 54s - loss: 0.5116 - accuracy: 0.7676 - val_loss: 0.4968 - val_accuracy: 0.7785 - 54s/epoch - 34ms/step
Epoch 8/15
1618/1618 - 57s - loss: 0.5039 - accuracy: 0.7707 - val_loss: 0.4938 - val_accuracy: 0.7802 - 57s/ep

### Making the predictions

In [73]:
sample_sub=pd.read_csv(r'C:\Users\DELL\Desktop\Kaggle Playground\Disaster prediction using Tweets\sample_submission.csv')

In [76]:
test

,id,keyword,text
0,0,NaN,Just happened a terrible car crash
1,2,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...
3258,10861,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


### Preprocessing the test-data, as we need to convert it into model-readable format.

In [77]:
test['input'] = test['text'].map(lambda x:pipeline(x))

In [78]:
test

,id,keyword,text,input
0,0,NaN,Just happened a terrible car crash,happen terrible car crash
1,2,NaN,"Heard about #earthquake is different cities, s...",hear earthquake different city stay safe
2,3,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese flee street save
3,9,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse light Spokane wildfire
4,11,NaN,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kill 28 China Taiwan
...,...,...,...,...
3258,10861,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,earthquake safety LOS ANGELES SAFETY fastener ...
3259,10865,NaN,Storm in RI worse than last hurricane. My city...,storm ri bad hurricane city amp 3others hard h...
3260,10868,NaN,Green Line derailment in Chicago http://t.co/U...,Green Line derailment Chicago
3261,10874,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,meg issue Hazardous Weather Outlook HWO


In [79]:
test_corpus = create_corpus(test)

In [86]:
test_sequences=tokenizer_obj.texts_to_sequences(test_corpus)
test_tweet_pad=pad_sequences(test_sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [87]:
test_tweet_pad.shape

(3263, 50)

In [89]:
test_tweet_pad

array([[ 200, 1621,   49, ...,    0,    0,    0],
       [ 137,  197,  985, ...,    0,    0,    0],
       [ 132,    5,  582, ...,    0,    0,    0],
       ...,
       [ 724,  480,  289, ...,    0,    0,    0],
       [5127,  187,  340, ...,    0,    0,    0],
       [1314, 1315,   19, ...,    0,    0,    0]])

In [91]:
word_index['car']

49

### Final prediction

In [92]:
y_pre = model.predict(test_tweet_pad)

102/102 [==============================] - 1s 4ms/step


In [93]:
type(y_pre)

numpy.ndarray

In [95]:
y_pre.shape

(3263, 1)

In [97]:
y_pre[0:5]

array([[0.8249626 ],
       [0.8091696 ],
       [0.86521316],
       [0.8212333 ],
       [0.8981525 ]], dtype=float32)

### Saving the file that can be directly submitted on kaggle.

In [100]:
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('submission_word_embeddings.csv',index=False)

In [99]:
y_pre

array([1, 1, 1, ..., 1, 0, 1])